## EKF Imeplementation

This Notebook contains codes to implement Extended Kalman Filter on a JetBot.

In [143]:
import os
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import Robot

robot = Robot()

In [1]:
# Mapping between set_motor "speed" and measured wheel angular velocity "omega"
# for 0.1 second motor running time
wheel_calibration = {
    "speed": [0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
    "omega": [0.0, 3.85, 9.23, 15.0, 25.8, 29.2, 35.4]
}

In [127]:
def forward(wheel_speed, Rtime):

    robot.set_motors(wheel_speed, wheel_speed)
    time.sleep(Rtime)
    robot.stop()
    
    return

def control2robot(wheel_radius, axle_length):
    """ transform wheel speeds to robot motion in world frame """
    l = axle_length
    r = wheel_radius

    return np.array([[r/2, r/2],
                  [r/l, -r/l]])

def omega2speed(in_val, mapping, debug=False):
    """ Map wheel angular speed to motor speed setting based on a calibration mapping """
    
    if in_val < 0:
        sign = -1
        in_val = abs(in_val)
    else:
        sign = 1
        
    out_lower = 0
    in_lower = 0
    out_val = 0

    for i, in_upper in enumerate(mapping["omega"]):
        if debug:
            print (i, in_upper)
        if in_val < in_upper:
            out_upper = mapping["speed"][i]
            out_val = out_lower + (in_val - in_lower)/(in_upper - in_lower) \
                *(out_upper-out_lower)
            if debug:
                print("yes", out_val)
            break
        else:
            if debug:
                print("no")
            out_lower = mapping["speed"][i]
            in_lower = in_upper
            
    if out_val is 0:
        print ("Input is too high!!!", in_val)
        out_val = 0
        
    return sign*out_val


def calc_wheel_velocities(direction='L', arc_radius=0.5, min_ang_vel=3.85, \
       wheel_radius=0.0325, axle_length=0.12, debug = False):
    """ Calculate wheel velocities to generate forward arc motion of provided radius """

    radius = arc_radius
    axle = axle_length
    
    if direction is 'L':
        """ If left turn, angular velocity of right wheel should be higher.
        Set angular velocity of left wheel to minumum (e.g. 3.85--> motor setting of 0.3) """
        l_ang_vel = min_ang_vel
        r_ang_vel = (min_ang_vel*2)/(2*radius/axle-1)+min_ang_vel
    else:
        """ If right turn, angular velocity of left wheel should be higher.
        Set angular velocity of right wheel to minumum (e.g. 3.85--> motor setting of 0.3) """
        r_ang_vel = min_ang_vel    
        l_ang_vel = (min_ang_vel*2)/(2*radius/axle-1)+min_ang_vel
    
    if debug:
        print ("Left angular velocity:",l_ang_vel, " Right angular velocity:",r_ang_vel)
        T = control2robot(wheel_radius, axle_length)
        robot_velocities = np.dot(T, np.array([[r_ang_vel],[l_ang_vel]]))
        print ("Robot velocities:", robot_velocities)
        print("arc radius = ",abs(robot_velocities[0,0]/robot_velocities[1,0]))
    
    return np.array([[r_ang_vel],[l_ang_vel]])


In [48]:
def take_circ_step(robot_params, direction, radius, debug=False):
    
    # load robot control parameters
    start_x = robot_params["start_x"]
    start_y = robot_params["start_y"]
    wheel_radius = robot_params["wheel_radius"]
    axle_length = robot_params["axle_length"]
    motor_on_time = robot_params["motor_on_time"]
    motor_off_time = robot_params["motor_off_time"]
    min_ang_velocity = robot_params["min_ang_velocity"]
    
    """ Generate clamped wheel velocities based on turn direction and radius """
    wheel_velocities = calc_wheel_velocities(direction='L', arc_radius=radius, \
        min_ang_vel=min_ang_velocity, \
        wheel_radius=wheel_radius, axle_length=axle_length, debug = debug)
    
    """ Map wheel angular velocities to motor setting, then run motors """ 
    w_r = omega2speed(wheel_velocities[0,0],wheel_calibration) 
    w_l = omega2speed(wheel_velocities[1,0],wheel_calibration)
    if debug:
        print ("L motor:", w_l," R motor:", w_r)
    
    """ Run motor step motion """
    robot.set_motors(w_l, w_r)  # left, right
    time.sleep(motor_on_time)
    robot.stop()
    time.sleep(motor_off_time)
    
    return wheel_velocities

In [47]:
robot_params = {
    # pose
    "start_x": 1.5,
    "start_y": 1.5,
    "start_theta": math.pi/2,
    # physical dimensions
    "wheel_radius": 0.0325,
    "axle_length": 0.12,
    # stepwise motor control
    "motor_on_time": 0.1,  
    "motor_off_time": 0.2,
    # wheel velocity control
    "min_ang_velocity": 6.0 # Equivalent to motor speed setting of 0.3
}



In [145]:
LARGE = 1e6
t_delta = 0.1  # motor on time
dt = 0.1
R_t = np.zeros((3,3))  # Assume zero control noise for now

debug = True
np.set_printoptions(precision=5)

wheel_radius = robot_params["wheel_radius"]
axle_length = robot_params["axle_length"]
T = control2robot(wheel_radius,axle_length)

# robot's initial pose
x = robot_params["start_x"]
y = robot_params["start_y"]
theta = robot_params["start_theta"]

# landmark's coordinates
landmarks = ['cup', 'horse']

# Initialize Mu
Mu = np.array([[x],[y],[theta]])
for object in enumerate(landmarks):
    Mu = np.vstack((Mu,np.array([[0],[0]])))
N = Mu.shape[0]   # N=3+2n, n=num of landmarks

# Initialize Sigma - For Σ_mm, infinity (large num) along the diagonal and zero everywhere else 
Sigma = np.zeros((N,N))
Sigma[3:,3:] = np.eye(N-3)*LARGE

# Take 1 circular step
wheel_velocities = take_circ_step(robot_params, 'L', 0.5, debug=False)
robot_velocities = np.dot(T,wheel_velocities)   # convert to (v,omega)

if debug:
    print(Mu)
    print(Sigma)
    print(wheel_velocities)
    print(robot_velocities)

    
""" EKF Prediction Step """
v = robot_velocities[0,0]
w = robot_velocities[1,0]
arc_radius = v/w   # arc radius 

x_delta = arc_radius*(math.sin(theta+w*t_delta)-math.sin(theta))
y_delta = arc_radius*(math.cos(theta)-math.cos(theta+w*t_delta))
theta_delta = w*t_delta

F = np.zeros((3,N))
F[0:3,0:3] = np.eye(3)

# Implement Mu_t = g(Mu_t-1, u_t) 
Mu = Mu + np.dot(F.T, np.array([[x_delta],[y_delta],[theta_delta]]))

# Calculate G_t
d_x_delta = arc_radius*(math.cos(theta+w*t_delta)-math.cos(theta))
d_y_delta = arc_radius*(-math.sin(theta)+math.sin(theta+w*t_delta))
G_x_t = np.array([[0,0,d_x_delta],[0,0,d_y_delta],[0,0,0]])
G_t = np.eye(N)+ np.dot(np.dot(F.T, G_x_t),F)

# Calculate Sigma_t = G_t.Sigma_t-1.G_t^T + F^T.R_t.F
Sigma = np.dot(np.dot(G_t, Sigma),G_t.T) + np.dot(np.dot(F.T, R_t),F)

if debug:
    print(v,w,arc_radius)
    print(x_delta,y_delta,theta_delta)
    print(F)
    print(Mu)
    np.set_printoptions(suppress=True)
    print(G_t)
    print(Sigma)

[[1.5   ]
 [1.5   ]
 [1.5708]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]]
[[      0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       0.       0.       0.       0.       0.]
 [      0.       0.       0. 1000000.       0.       0.       0.]
 [      0.       0.       0.       0. 1000000.       0.       0.]
 [      0.       0.       0.       0.       0. 1000000.       0.]
 [      0.       0.       0.       0.       0.       0. 1000000.]]
[[7.63636]
 [6.     ]]
[[0.22159]
 [0.44318]]
0.22159090909090912 0.4431818181818181 0.5000000000000001
-0.0004909449465606009 0.022151837838428445 0.04431818181818181
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]]
[[1.49951]
 [1.52215]
 [1.61511]
 [0.     ]
 [0.     ]
 [0.     ]
 [0.     ]]
[[ 1.       0.      -0.02215  0.       0.       0.       0.     ]
 [ 0.       1.      -0.00049  0.       0.       0.       0.     ]
 [ 0.       0. 

In [ ]:
# Make approx 3 to 4 circular runs - 300 steps
for i in range(3):
    take_circ_step(robot_params, 'L', 0.5, debug=False)

In [30]:
robot.stop()

In [122]:
np.set_printoptions(suppress=False)

In [105]:
Mu

array([[1.49950906],
       [1.52215184],
       [1.61511451],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ]])

In [106]:
Mu += np.dot(F.T, np.array([[x_delta],[y_delta],[theta_delta]]))

In [142]:
np.dot(np.dot(F.T, R_t),F)

array([[0.00003, 0.0005 , 0.005  , 0.     , 0.     , 0.     , 0.     ],
       [0.0005 , 0.01   , 0.1    , 0.     , 0.     , 0.     , 0.     ],
       [0.005  , 0.1    , 1.     , 0.     , 0.     , 0.     , 0.     ],
       [0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ]])

In [141]:
Sigma = np.zeros((N,N))
Sigma[3:,3:] = np.eye(N-3)*LARGE
np.dot(np.dot(G_t, Sigma),G_t.T)

array([[      0.,       0.,       0.,       0.,       0.,       0.,
              0.],
       [      0.,       0.,       0.,       0.,       0.,       0.,
              0.],
       [      0.,       0.,       0.,       0.,       0.,       0.,
              0.],
       [      0.,       0.,       0., 1000000.,       0.,       0.,
              0.],
       [      0.,       0.,       0.,       0., 1000000.,       0.,
              0.],
       [      0.,       0.,       0.,       0.,       0., 1000000.,
              0.],
       [      0.,       0.,       0.,       0.,       0.,       0.,
        1000000.]])